In [ ]:
# abalone 데이터셋

# abalone # 4177건
# 7:3 비율로 train set / test set 데이터를 분할(random state = 20201005)
# train set -> 모델 -> test set -> 나이를 예측(rings) => rmsle 적용 => 점수 출력
# random forest regressor 이용
# min max scaling

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

In [7]:
abalone = pd.read_csv("abalone.txt", header = None, names = ['sex', 'length', 'diameter',
                                                             'height', 'whole_weight',
                                                             'shucked_weight', 'viscera_weight',
                                                             'shell_weight', 'rings'])
abalone

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [8]:
sexMapping = {
    'M' : 0,
    'F' : 1,
    'I' : 2
}

In [9]:
abalone['sex'] = abalone['sex'].map(lambda x : sexMapping.get(x))
abalone

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,2,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,1,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,0,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,0,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,1,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [13]:
x = abalone.iloc[:, :-1]
y = abalone['rings']

In [25]:
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.3, random_state = 20201005)

In [15]:
def rmsle(pv, av) : # 예측값, 실제값
    # 넘파이 배열로 변환
    pv = np.array(pv)
    av = np.array(av)
    
    # 예측값과 실제값에 1을 더하고 로그를 씌운다
    log_predict = np.log(pv + 1)
    log_actual = np.log(av + 1)
    
    res = log_predict - log_actual
    res = np.square(res)
    
    mean_res = res.mean()
    score = np.sqrt(mean_res)
    return score

In [16]:
rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

In [17]:
kfold = KFold(n_splits = 10, shuffle = True, random_state = 20201005)

In [93]:
model = RandomForestRegressor(n_estimators = 250,
                              n_jobs = -1,
                              max_depth = 10,
                              min_samples_split = 10,
                              random_state = 20201005)

In [94]:
model.fit(xTrain, yTrain)

RandomForestRegressor(max_depth=10, min_samples_split=10, n_estimators=250,
                      n_jobs=-1, random_state=20201005)

In [95]:
round(model.score(xTrain, yTrain)*100, 2)

78.21

In [96]:
score = cross_val_score(model, xTrain, yTrain, cv = kfold, scoring = rmsle_scorer)
score

array([0.18033318, 0.17772929, 0.17585265, 0.18230415, 0.19123056,
       0.17465085, 0.16524391, 0.19634   , 0.17439392, 0.14976022])

In [97]:
score.mean()

0.17678387286964697